In [1]:
from PIL import Image
import urllib.request
import io
from skimage import io
from ipywidgets import Image
import pandas as pd
import time 
import ast
from all_functions import *
import requests

In [2]:
 

# google maps api: 
APIkey = "YOUR API KEY"

In [3]:
def get_address(query):
        base = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
        locationbias= '42.3314584645106, -71.1039191294055'
        key = APIkey
        query = query
        fields = 'formatted_address,name,geometry'
        inputtype = 'textquery'
        
        url_search = base + query + '&inputtype=' + inputtype + '&fields=' + fields + '&locationbias=circle:2000@' + locationbias + '&key=' + key
        my_dict = requests.get(url_search).json()
        return my_dict

In [4]:
def get_commute(dest, origin):
    
    base = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
    origin = 'origins=' + origin + '&'
    dest_str = 'destinations=' + dest + '&'
    api_key = 'key=' + APIkey
    
    url_search = base + origin + dest_str + api_key
    my_dict = requests.get(url_search).json()
    return my_dict

In [88]:
def ult_scrap(results, city, job_title, site_list):
    file = 1
    SKIPPER = 0 # Skips this many of the first jobs
    job_col_list = ['site', 'title', 'company', 'location', 'salary', 'summary', 'tags','id', 'comp_loc', 'link', 'pulldate','postdate', 'ezapply']
    job_df = pd.DataFrame(columns=job_col_list, index=range(0,results))
    comm_df = pd.DataFrame(columns=['company', 'place', 'address', 'distance', 'duration'])
    #if(file > SKIPPER):
        # for results
    num = 0
    for site in site_list: 
        site_ind = 0
        
        if site == 'stack':
            list_o_divs = []
            # https://stackoverflow.com/jobs?q=data+scientist&l=boston&d=20&u=Miles
            pg = 1
            base = 'https://stackoverflow.com/jobs?'
            tot_pg = ((results // 20) + 1)
            while pg <= tot_pg:
                params = {'q' : job_title, 'sort' : 'p', 'l' : city, 'd' : '20', 'u': 'Miles', 'pg': pg}
                page = requests.get(base, params)
                time.sleep(1)  
                soup = get_soup(page.text)
                divs = soup.find_all(name="div", attrs={"class":"-job-summary"})
                list_o_divs.append(divs)
                pg = pg + 1
            
        if site == 'indeed':
            list_o_divs = []
            # https://www.indeed.com/jobs?q=data+science&l=boston&sort=date
            res_left = results
            base = 'https://www.indeed.com/jobs?'
            while res_left > 0:            
                params = {'q' : job_title,'l' : city, 'start' : (results-res_left),  'sort' : 'date', 'radius': '25'}
                page = requests.get(base, params) 
                time.sleep(1)  
                soup = get_soup(page.text)
                divs = soup.find_all(name="div", attrs={"class":"row"})
                list_o_divs.append(divs)
                res_left = res_left - len(divs)
            
        if site == 'linkedin':
            '''url = 'http://www.linkedin.com/jobs/search/?keywords=data+science&location=Boston&sortBy=DD'
            base = 'http://www.linkedin.com/jobs/search/?'
            params = {'keywords' : job_title, 'location' : city, 'sortBy' : 'DD'}
            
            s=requests.session()
            HOMEPAGE_URL = 'https://www.linkedin.com'
            LOGIN_URL='https://www.linkedin.com/uas/login-submit'

            html = s.get(HOMEPAGE_URL).content
            soup = BeautifulSoup(html, "lxml")
            csrf = soup.find(id="loginCsrfParam-login")['value']
            payload={
                 'session_key':'YOUREMAIL',
                'session_password':'YOURPASS',
                'loginCsrfParam': csrf
            }

            s.post(LOGIN_URL,data=payload)
            r = s.get(url)
            soup = BeautifulSoup(r.content, 'lxml')'''
            #divs = soup.find_all(name="div", attrs={"class":"job-card-search__content-wrapper"})

        #if(len(divs) == 0):
            #break
            
        for divs in list_o_divs:
            for div in divs:
                job_df.loc[num, 'sID'] = 'ID' + str(num)
                job_df.loc[num, 'site'] = site
                job_df.loc[num, 'pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                job_df.loc[num, 'link'] = extract_link(div, site)
                job_df.loc[num, 'title'] = extract_job_title(div, site)
                job_df.loc[num, 'summary'] = extract_summary(div, site)
                job_df.loc[num, 'salary'] = extract_salary(div, site)
                job_df.loc[num, 'location'] = extract_location(div, site).strip()
                job_df.loc[num, 'tags'] = extract_tags(div, site)
                job_df.loc[num, 'id'] = extract_id(div, site)
                job_df.loc[num, 'ezapply'] = extract_easyapply(div, site)
                job_df.loc[num, 'postdate'] = extract_postdate(div, site)
                
                if site =='indeed':
                    job_df.loc[num, 'company'] = extract_company(div, site).strip()
                    job_df.loc[num, 'comp_loc'] = str(extract_company(div, site).strip() + ', ' + extract_location(div, site).strip())
                if site == 'stack':
                    job_df.loc[num, 'company'] = extract_company(div, site).split('-')[0].strip()
                    job_df.loc[num, 'comp_loc'] = extract_company(div, site)
                #test = extract_easyapply(div, site)
                num += 1
    job_df = job_df.set_index('sID')
    companies = job_df['comp_loc']
    ids = job_df.index.tolist()

    
    for index, company in enumerate(companies):
        company = company
        tmp_id = ids[int(index)]
        text_dict = get_address(str(company))
        if text_dict['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_address.')
            continue
        else:
            address = text_dict['candidates'][0]['formatted_address']
            place = text_dict['candidates'][0]['name']    
            res_lat_lng = str(text_dict['candidates'][0]['geometry']['location']['lat']) + ',' + str(text_dict['candidates'][0]['geometry']['location']['lng'])

        comm = get_commute(dest=res_lat_lng, origin='8 Oswald Street')
        if comm['rows'][0]['elements'][0]['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_commute.')
            continue
        else:
            try:
                distance = (comm['rows'][0]['elements'][0]['distance']['text'])
                duration = (comm['rows'][0]['elements'][0]['duration']['text'])
            except:
                print(comm)
                break


        comm_df.loc[index, 'company'] = company
        comm_df.loc[index, 'place'] = place
        comm_df.loc[index, 'address'] = address
        comm_df.loc[index, 'distance'] = distance
        comm_df.loc[index, 'duration'] = duration
        comm_df.loc[index, 'sID'] = tmp_id
        
    comm_df.set_index('sID', inplace = True)
    #job_df['id'] = job_df['id'].astype('str')
    fin_df = job_df.merge(comm_df, how = 'left', left_index=True, right_index=True)#.drop('company_y',1)
    file = file + 1
    print('job - fin qual check:' + str(len(job_df) - len(fin_df)))
    #fil_df = fin_df.groupby('id').first()
    
    return fin_df

In [89]:
fetch = ult_scrap(50,  job_title= 'data+science', city='boston', site_list=['indeed', 'stack'])
fetch

Quest Groups LLC, Boston, MA has 0 results in get_address.
ExperienceFlow, Boston, MA has 0 results in get_address.
ExperienceFlow, Boston, MA has 0 results in get_address.
Versa Shore Inc., Boston, MA 02137 (Hyde Park area) has 0 results in get_address.
Quest Groups LLC, Boston, MA has 0 results in get_address.
Quest Groups LLC, Boston, MA has 0 results in get_address.
ExperienceFlow, Boston, MA has 0 results in get_address.
The Hagan-Ricci Group, Inc., Boston, MA has 0 results in get_address.
ExperienceFlow, Boston, MA has 0 results in get_address.
Quest Groups LLC, Boston, MA has 0 results in get_address.
UnitedHealth Group, Boston, MA 02298 (Central area) has 0 results in get_address.
Armored Things             - Boston, MA         has 0 results in get_address.
Armored Things             - Boston, MA         has 0 results in get_address.
Grassroots Technologies, Inc             - Boston, MA         has 0 results in get_address.
Plannuh             - Boston, MA         has 0 results

,site,title,company_x,location,salary,summary,tags,id,comp_loc,link,pulldate,postdate,ezapply,company_y,place,address,distance,duration
sID,,,,,,,,,,,,,,,,,,
ID0,indeed,Data Scientist for Scientific Visualization vi...,HiTS,"Boston, MA 02115 (Fenway-Kenmore area)",Unspecified,Qualification in data science. She or he will ...,Tags on Stack Overflow,p_ea06f7880b712ca7,"HiTS, Boston, MA 02115 (Fenway-Kenmore area)",http://www.indeed.com/rc/clk?jk=ea06f7880b712c...,10-29-2018_04-06_PM,Just posted,True,"HiTS, Boston, MA 02115 (Fenway-Kenmore area)",The Inn At Longwood Medical,"342 Longwood Ave, Boston, MA 02115, USA",0.9 mi,6 mins
ID1,indeed,Curriculum Developer - DataRobot University,DataRobot,"Boston, MA 02109 (Central area)",Unspecified,"Instructional experience, preferably in data s...",Tags on Stack Overflow,p_1b068bba6b7d022e,"DataRobot, Boston, MA 02109 (Central area)",http://www.indeed.com/rc/clk?jk=1b068bba6b7d02...,10-29-2018_04-06_PM,Just posted,True,"DataRobot, Boston, MA 02109 (Central area)","DataRobot, Inc.","1 International Place, 5th Floor, Boston, MA 0...",5.1 mi,16 mins
ID2,indeed,"Analyst, Experiences and Rentals",TripAdvisor,"Needham, MA",Unspecified,Find reporting and self-service data solutions...,Tags on Stack Overflow,p_59d0916e71049446,"TripAdvisor, Needham, MA",http://www.indeed.com/rc/clk?jk=59d0916e710494...,10-29-2018_04-06_PM,Just posted,False,"TripAdvisor, Needham, MA","TripAdvisor, Inc.","400 1st Ave, Needham, MA 02494, USA",7.7 mi,25 mins
ID3,indeed,Lead Data Scientist,Daley and Associates,"Boston, MA 02111 (Central area)","$220,000 a year",You will ensure data quality throughout all st...,Tags on Stack Overflow,p_2b0982ad1a1f6c6d,"Daley and Associates, Boston, MA 02111 (Centra...",http://www.indeed.com/rc/clk?jk=2b0982ad1a1f6c...,10-29-2018_04-06_PM,Just posted,True,"Daley and Associates, Boston, MA 02111 (Centra...","Daley And Associates, LLC","1 Financial Center, Boston, MA 02111, USA",3.7 mi,18 mins
ID4,indeed,Software Eningeer for Web Application Development,HiTS,"Boston, MA 02115 (Fenway-Kenmore area)",Unspecified,"The LSP houses students, fellows and staff act...",Tags on Stack Overflow,p_45213e2c5e26454e,"HiTS, Boston, MA 02115 (Fenway-Kenmore area)",http://www.indeed.com/rc/clk?jk=45213e2c5e2645...,10-29-2018_04-06_PM,Just posted,True,"HiTS, Boston, MA 02115 (Fenway-Kenmore area)",The Inn At Longwood Medical,"342 Longwood Ave, Boston, MA 02115, USA",0.9 mi,6 mins
ID5,indeed,Strategic Sales Analyst,Babcock Power Services,"Marlborough, MA",Unspecified,"Focus in Finance, Economics or Data Science pr...",Tags on Stack Overflow,p_faad3393de121f43,"Babcock Power Services, Marlborough, MA",http://www.indeed.com/rc/clk?jk=faad3393de121f...,10-29-2018_04-06_PM,Babcock Power Inc.,False,"Babcock Power Services, Marlborough, MA",Babcock Power Services Inc,"26 Forest St Suite 300, Marlborough, MA 01752,...",35.6 mi,46 mins
ID6,indeed,Lead software engineer in cloud-based data man...,HiTS,"Boston, MA 02115 (Fenway-Kenmore area)",Unspecified,Background in data management. The LSP houses ...,Tags on Stack Overflow,p_df0c41baa5ab39a7,"HiTS, Boston, MA 02115 (Fenway-Kenmore area)",http://www.indeed.com/rc/clk?jk=df0c41baa5ab39...,10-29-2018_04-06_PM,Just posted,True,"HiTS, Boston, MA 02115 (Fenway-Kenmore area)",The Inn At Longwood Medical,"342 Longwood Ave, Boston, MA 02115, USA",0.9 mi,6 mins
ID7,indeed,Full-Stack Engineer (Front and/or Back-end),Eagle Investment Systems,"Wellesley Hills, MA 02481","$90,000 - $150,000 a year",Eagle is a startup in a big company body - we ...,Tags on Stack Overflow,p_d3b189effac95af9,"Eagle Investment Systems, Wellesley Hills, MA ...",http://www.indeed.com/company/Eagle-Investment...,10-29-2018_04-06_PM,Today,True,"Eagle Investment Systems, Wellesley Hills, MA ...",Eagle Investment Systems LLC,"45 William St, Wellesley, MA 02481, USA",7.2 mi,21 mins
ID8,indeed,Hadoop Engineer/Administrator,Eagle Investment Systems,"Wellesley Hills, MA 02481","$140,000 - $175,000 a year",Al